## Life Expectancy Research
**STAT 301  Project Proposal - Group 1  
Date:         November 1, 2022**

## Introduction




## Preliminary Results

### Data Importing, Cleaning, and Wrangling
  

## Methods - Plan



## Reference
  
  